# IBM Cloud Hyper Protect Virtual Servers Secure Build and Bring Your Own Image lab (with Docker Hub)

This lab will introduce you to two features of the IBM Cloud Hyper Protect Virtual Servers service:

1. Secure Build
2. Bring Your Own Image (BYOI)

The initial introduction of **_IBM Cloud Hyper Protect Virtual Servers_** (HPVS) offered an Ubuntu image which you could then tailor to your requirements, but if you wanted to replicate your changes onto other HPVS instances, you would have to repeat the custom configurations on each additional HPVS instance you created.  Automation tools like Ansible could help with that process, but wouldn't it be nicer if you could just deploy your own custom-built Docker image, that you only had to configure once, onto an HPVS instance?  That's the convenience that the *BYOI* feature offers. It offers the ability to deploy a digitally signed Docker image into an HPVS instance.

You can deploy Docker images that are digitally signed by you or some other person or organization that you trust. In the case of images you build and sign, the _Secure Build_ feature lets you build, and digitally sign, a Docker image within an HPVS instance running an IBM-provided _secure build server_ that offers the confidentiality, integrity and auditability protections inherent in IBM Cloud Hyper Protect Virtual Servers.  

In this lab you will perform the main use case targeted by these two features working together:

1. You will deploy the secure build server in your own HPVS instance.
2. You will use this secure build server to securely create and digitally sign a Docker image that contains a sample application. For this lab you will use a sample _secure bitcoin wallet application_ provided by IBM for demonstration purposes.
3. You will deploy this Docker image that you've built in a second HPVS instance and then run the sample secure bitcoin wallet application contained within the image. You will configure your HPVS instance to use an **_IBM Cloud Hyper Protect Crypto Services_** (HPCS) instance for key generation.  The HPCS instance provides the security of a _FIPS 140-2 level 4_-certified _Hardware Security Module_, which is the strongest level of FIPS 140-2 certification commercially available.

You will be using an open-source application called _Jupyter Notebook_ to run this lab. Jupyter Notebook allows you to enter commands and see the results of these commands. You are likely reading this within Jupyter Notebook right now.  You will also open a terminal within Jupyter and enter some commands from the terminal. The large majority of the commands will be run from within the Jupyter notebook while a small number of commands will be run from the terminal. Those commands that prompt for user input are run from the terminal, while the vast majority, which don't prompt for user input, are run from within Jupyter Notebook.

## Secure Build overview

The diagram below provides an overview of the major components involved in the secure build process that you will be working with in this lab. An explanation of each numbered annotation is provided below the diagram.

![Secure Build architecture](images/sbsarch.png)

1. The IBM Cloud command line interface (CLI) will be used throughout this lab.  It has already been installed for you in the lab environment.

2. You will be using your IBM Cloud account during the lab.

3. The _secure build_ CLI is a separate CLI used exclusively for the Secure Build process.  You will download this CLI in _Step 5.1_ of this lab and will use it throughout the remainder of the lab.

4. You will be building a container image from an application whose source code is in a GitHub repository. The GitHub repository is specified for you in the configuration file you will build in _Step 5.5_ of this lab.  It is the value of the _GITHUB\_URL_ key given for you in _Step 5.5_. 

5. The container image that you build will be pushed to the _Docker Hub_.

6. The secure build server runs inside a Hyper Protect Virtual Servers instance in IBM Cloud that you will create in _Step 5.11_ of the lab.

7. Your secure build server contains a snapshot of its state.  You can download this _secure build state_ from the server and use it to rebuild the secure build server if necessary.  Although we won't be using this downloaded state in the lab, you will perform the command to download it in _Step 6.10_ of the lab.

**Note:** It is also possible to save the _secure build state_ to _IBM Cloud Object Storage_, but we will not be doing that in this lab.

Okay, let's get started!!

## Initial login to IBM Cloud via the _ibmcloud_ command line interface (CLI)

### Step 1.1 Open a terminal with the Jupyter Launcher

In the browser tab in which you are accessing the Jupyter Notebook to take this lab, you should see a workspace with two tabs:

    1. The tab you are using right now to read this lab.
    2. The Jupyter launcher tab.

Go to the Jupyter Launcher tab.  Click the *Terminal* tile at the bottom left and the Launcher tab will be replaced with a terminal window that you will use to enter a small number of commands from this lab that require interactive user input.

![Choose Terminal](images/opennewterminaliniksd.png)

When you have done that, come back here to continue with the instructions.

---

### Step 1.2 Use the terminal tab to log in to IBM Cloud

Navigate to your terminal tab, and from there, enter the command `ibmcloud login -r YOUR-REGION --sso`, substituting one of the following regions for YOUR-REGION:  `us-east`, `us-south`, `au-syd` or `eu-de`, which are for United States (East), United States (South), Sydney, Australia, and Frankfurt, Germany, respectively.  

For your convenience the proper command for each region is listed. From the four commands listed below, copy the one applicable for your region, and paste it into the terminal tab and run the command:

`ibmcloud login -r us-east --sso`

`ibmcloud login -r us-south --sso`

`ibmcloud login -r au-syd --sso`

`ibmcloud login -r eu-de --sso`

---

### Step 1.3 Follow the prompts to complete the login

Follow the instructions to log in to your IBM Cloud account.  The login process from the terminal should be like this:

You will first be given a *Y/n* prompt to open the URL to get a one-time passcode, reply *n*. The reason to reply *n* is because in your lab environment your server does not have graphics capabilities and cannot open the URL.

In order to overcome this, you must copy the URL given in the command output and paste it into another browser tab or window. The goal is to obtain a temporary one-time code for you.
In order to get this code to use to log in with the CLI, you may need to log in to the IBM Cloud Web UI with your browser. If so, follow any necessary login prompts. You should eventually be given a page with a one-time code.  Click on that code to put it in your clipboard, and paste it at the input prompt back in your terminal tab.

When successful, your terminal window will look like this, but with your account information of course: 

```

API endpoint:      https://cloud.ibm.com
Region:            us-east
User:              silliman@us.ibm.com
Account:           Barry Silliman's Account (1e963a246cc69a44df65e277e14239d5) <-> 1996902
Resource group:    No resource group targeted, use 'ibmcloud target -g RESOURCE_GROUP'
CF API endpoint:
Org:
Space:
```

When that is complete, come back to this tab and continue in this notebook. Leave the terminal tab open, you will need it later.

---

### Step 1.4 Target the default resource group

---

#### Jupyter Notebook tip

The text you are reading now is in a type of Jupyter notebook cell called _markdown_.  It is intended for instructions, and includes text and possibly images.

Commands that you will enter (with the exception of the small number of commands, such as the `ibmcloud login -r YOUR-REGION --sso` above, that are run in the terminal tab) will be in a second type of Jupyter notebook cell called _code_.  At the top of this page you can see a dropdown box that will show you whether or not you are in a markdown cell or a code cell. The below screen snippet shows where this dropdown box is located:

![Choose Terminal](images/celltypeindication.png)

---

You can see from the output from _Step 1.3_ that no _resource group_ is targeted when you log in. The command in the code cell below will target the _default_ resource group. Resource groups are logical buckets that you can define in order to categorize your IBM Cloud resources as you wish.  We will simply use the _default_ resource group which exists already.

Click on the code cell below that contains the command `ibmcloud target -g default`. You should see _code_ in the dropdown box at the top. 

Now that you have the code cell below highlighted, click the _Run_ button from the menu above—it is the triangle icon—and that command will be run, and its output will be shown.

This will target your _default_ resource group. See the debugging tip below the code cell if this command does not work for you.

In [ ]:
! ibmcloud target -g default

#### Debugging tip

Some IBM Cloud users may not have a _default_ group but instead have a _Default_ group, with an uppercase _D_.  If the above command fails, try using *Default* instead of *default*.  You can do that by just clicking in the code cell above and changing the _d_ to _D_ in default.  Then run the code cell again. If that still doesn't work, ask an instructor for help.

---

### A note about running Jupyter Notebook code cells

Observe that to the left of the code cells are square brackets with empty contents. When you run a code cell, an asterisk (*\**) appears between the square brackets.  (If the command runs fast enough you may not even notice the appearance of the asterisk). When the command finishes, a number appears between the square brackets.  During this lab, some of the commands may take several seconds to run, but no command in this lab should take too long. Each time that you run a code cell, ensure that you wait for that code cell to complete execution before continuing with the lab. If the asterisk remains in between the square brackets for more than thirty seconds, ask an instructor for help—none of the commands in this lab should take that long.

The number shown between the square brackets upon command completion is incremented by one each time you run a code cell.  The numbers are helpful in ensuring that you have run each code cell in the correct order.

---

## Create an ssh key for the lab and add it to your GitHub account

### Step 2.1 Create an ssh key pair in your lab system

As part of this lab, you will need to create a key pair that you will then define in your personal GitHub account settings so that this key pair can be used to clone GitHub repositories.

**Note:** If you are an experienced GitHub user and already have ssh keys defined to your GitHub account, you should still create a new key for this lab and remove it from your account when the lab is over, for security reasons. **What is the security concern?** The lab instructors have access to your lab system in order to be able to help with debugging.  If the lab instructors were malicious, they could copy your ssh key, including the private key portion of the key pair.  They are not malicious—they are well-intentioned and kind. But to be completely secure, you should create a new key per the instructions in this step, and then remove it from your GitHub account when you have completed the lab. (The cleanup section of this lab helps you with the key removal from your GitHub account).

Run the code cell below to create a key pair in your lab workspace.

In [ ]:
! ssh-keygen -t rsa -b 4096 -f '/home/jovyan/.ssh/id_rsa' -N ''

---

### Step 2.2 Copy your public key to the clipboard and add it to your GitHub account

The above command created a key pair (a private part that you keep secret and a public part that can be shared safely with the world).  You need to add the public part to your GitHub account settings, so that this key pair can be used to do a clone of the GitHub repository of the sample secure bitcoin wallet application from within the Secure Build Server container.

Run the code cell below to list the public key, and copy this public key into your clipboard:

In [ ]:
! cat ~/.ssh/id_rsa.pub

Copy all of the output from the above `cat` command into the clipboard, and go to [this link](https://github.com/settings/ssh/new) to add this key to your GitHub account. When you get to that page (you may have to log in to GitHub first), put anything you want in the _Title_ field and paste the above output (which is your public key) into the _Key_ field and then click *Add SSH key*.  

**Note:** if you use a password manager and need to use copy and paste to log in to your GitHub account, go ahead and log in and then copy and paste the above output.

---

### Step 2.3 Add the GitHub public keys to your _known\_hosts_ file

Run the following code cell which gathers the public SSH keys of github.com and puts them in your lab workspace's _known\_hosts_ file. This is done only to eliminate a prompt about trusting the github.com host in the code cell after this one.

In [ ]:
! ssh-keyscan -H github.com >> "${HOME}/.ssh/known_hosts"

---

### Step 2.4 Verify you have added the key to your GitHub account correctly

Run the code cell below to verify that you have correctly added your SSH key pair to your GitHub account.  You should receive a message that says _Hi \<your GitHub username\>! You've successfully authenticated, but GitHub does not provide shell access._  If you received this message, continue in the lab.  If not, ask an instructor for help.  

In [ ]:
! ssh -T git@github.com

---

### Step 2.5 Create a personal access token for your GitHub account

The build of the sample secure bitcoin wallet application requires that a GitHub personal access token be provided to it. Even if you already have a GitHub personal access token, you are advised to create a new one for this lab. Log in to your GitHub account — you may still be logged in from _Step 2.2_ — and follow steps 1-9 (i.e., you can skip step 10) in [these instructions](https://docs.github.com/en/github/authenticating-to-github/creating-a-personal-access-token#creating-a-token) in the _Creating a token_ section. For step 8, when selecting scopes, you only have to select the first check box that says _repo_. This will also select the subitems in the _repo_ section—leave these subitems checked as well.

Ensure that you save the value of your newly created personal access token, as you will need it later on in the lab.

---

## Create a Docker Hub account if you do not already have one

You will need a Docker Hub account in order to do this lab, because the secure image that you build will be stored in Docker Hub under your account.

### Step 3.1 Sign up for a free Docker Hub account if necessary

If you already have a Docker Hub account, you can skip this step.

Sign up for a free Docker Hub account by going to [https://hub.docker.com/signup](https://hub.docker.com/signup) and following the instuctions.


---

## Create a Docker Access Token

In order to store the image that you will build into your Docker Hub account, you will need a Docker Access Token. This section will guide you through this process.

You could use your Docker Hub account password instead of a Docker Access Token, but this is not a good practice. By providing a Docker Access Token that you create just for this lab, you can delete this token at the end of the lab- thus you will not have to change your password to remain secure.

### Step 4.1 Log in to your Docker Hub account

If you are not yet logged in to your Docker Hub account, click [https://hub.docker.com](https://hub.docker.com) and click the **Sign In** link.
Follow any instructions or prompts necessary to complete the signin process.

### Step 4.2 Create a Docker Access Token

Follow [these instructions](https://docs.docker.com/docker-hub/access-tokens/#create-an-access-token) to create a new Docker Access Token.
Save the value of your access token in a safe place- you will not be able to retrieve the value later.



---

## Create your Secure Build Server HPVS instance

### Step 5.1 Download the Secure Build command line interface (CLI):

In order to set up your own Secure Build Server within an HPVS instance, you will use a command line interface (CLI) that is provided by IBM.  This CLI is implemented in Python and is publicly available (including source code) in a GitHub repository, which you will download now into your lab environment when you run the following code cell:

In [ ]:
! git clone git@github.com:ibm-hyper-protect/secure-build-cli.git

---

### Step 5.2 Change the working directory for the remainder of the lab

Up to now, your notebook has been running from the _/home/jovyan/labdir_ directory. For the remainder of the lab, we want to work from the directory that was just created by the _git clone_ command from the prior code cell.  Our new desired working directory is a subdirectory of our current working directory.

Run the next code cell in order to change directories to the _secure-build-cli_ directory that was created by the `git clone` command that you just ran.

In [ ]:
%cd secure-build-cli

---

### Step 5.3 Install Python packages required by the Secure Build CLI

There are some python packages required by the Secure Build CLI, and running the following code cell will install these packages using *pip3*, the Python 3.x-based package installer:

In [ ]:
! pip3 install -r requirements.txt

---

### Step 5.4 Create the Secure Build Server registration file

Part of the "BYOI" workflow involves creating a _secure registration file_, which contains secrets and metadata that can only be decrypted by the running image.  When you build an image later on in the lab, you will download a secure registration file for the image you will have built, and you will pass this file as an argument to the _ibmcloud hpvs_ command that you will use to create an HPVS instance built from your new image. 

The Secure Build Server itself is deployed securely within an HPVS instance with the help of its own secure registration file.  The code cell below will place this file into your lab environment workspace with the name `secure_build.asc`.

This file is posted publicly on the public documentation page for IBM Cloud Hyper Protect Virtual Servers, but the Secure Build CLI repository on GitHub does not contain it.  That's why you are creating it now.  This file is encrypted, and only the Secure Build Server can decrypt it.  IBM Cloud administrators cannot access your secure build server at all, and you can only access it through the Secure Build CLI, which limits your access to specific functions. Therefore, by design, the decrypted contents of this file cannot be obtained by any human.

**Note:** The code cell below will not produce any output, because it copies the encrypted message into a file.  But you know the command ended if a sequence number is placed within the square brackets to the left of the code cell. 

In [ ]:
%%bash
cat <<EOF > secure_build.asc
-----BEGIN PGP MESSAGE-----
Comment: https://gopenpgp.org
Version: GopenPGP 2.4.10

wcFMAxpkEC0f/4+uARAAmcGaAmxpBy+0p1gkcN2kHPLF7TW4wY7BHCLWolD30CP8
usNVbFcwxiDytoAu6xhLb2Z9IbF4cHgbv/k04kCFOm3hlI2cqkDLfHxwCDYvG8uC
6pb8G1KLQfEpaOBjJQweRPp2K5S8AfwGukDB5jyat2MR4ONQRVjPKSPJaf6L0gh5
RJoXnMBxBfbyeB/3KCze4pzPm5oHNVjkEKN1+y13XLzYI7EaltdZ3CDLVGNVl105
Dt9ksU7p/tAsRuysbwSXJ1tKn95oKpIAzVZnC5iXqs7UfjcDAcvvFv2KGhXh0L6Y
RPtd0hnf19vORD3EDsSxhKzIGzh9DZzLsVxR0DlnsUzDtE3F/XdudDsvIo1F8ocT
5kJmi0ScZtH2FRqweruQNcicO3/vz+BjM8hrRdkXfo24zPJc2D7jZooy2sCkfQ46
MKFjynIP1C+Yl9sNd1vPI/8aRYZVe7wQvMnVnd3bMHyoWxqvyVIBtaMr8XI8KDpy
rtNYw27OYxVWHKCfoJzMlsT6UfLqG1yh35f5eqovF8FAkDaITPmwkUKc1nxXH319
yfWwMY13NGBxie63tN0Re/4ANCmXTx6mAcASgG4i8LCRVDISwgyOGz9BhD1hFfhu
7mTh86LcDOKY4X5IE2g0fITThebooRKgTiTsr0yHApLvIadd+VJAEYSkjYxBRBzS
7QGJLk4dfUK1qKkuVAGqCmmnzvNcq4wJL/X2DnLAc55eWGkucDV3p4OUgtCLndeH
ajdBIgYyqFuz2D3kHlBrzWJ/ArTuTbtxNdCCabgp4VrITZGjHzr+scwGs55htrck
rv/AdrKzty2auiUXqanWgr2x8eCNrevq1Mcmt5qDmridWa7GvNSehC0G/yhc4UZJ
HGinc5LiuvuxN/5wwRnGMEhBCCdDw1M1pUNjgy3WUe4j0U+E6pST3TUREQzGgtws
0if/Yd+8YhZRUh6vCuwClXQ0GzYaqQr6dwtfDFgZi1ZbPu9vZP09Dz8b3nTKy7mF
46hvTyweayJJfXTN3+uUTcUtBOcdDXB66pEXfWzP7mr39nE+8nlq97NPP4dKYWpj
WJ7JU2/f9Qnfsah0qtMZFLcgcJ6V3Ty7u6idJT6ASVwJpolG12FMFycxeA1j9hUa
mN1IL8ZNcHi8Mst5A5aKQv8iFB0zXe7UYeteVRzVLwoYkiPz7iV/TIHirAL0JxB7
rrl9CifSBeqtlDgcOnsv/WO8b6sm19r2cf4GYe+ebqM9QHL0tytGWIiJFXzW7gyo
F1e01Qdj73/S0CY3/2qmfyMwmCyJqYzI1zTTCruj8yJxIKKh3uC36JEcxJgBKl4S
lzvhU4ITv+WEGM77itP7ukELB4cT+EHpy2spD7/JbkRTB4TlPoNDlBbYhfaQvjG5
r7NQOq4xv8Brk6q60zBp0gbgjL4TUwjQDcRA4PusEdBmW5UcD728N5IIHIeWPlb3
CxnsB0zJbpoWxeDVkU6Id48NZBzSIDKyyusE14j12CwEmJwih35/4x785/4/vqWF
HdSYiNF/V/2Tmro3DOMe97B7VxsDNO7xp8fejEbszJorE+wHmdDuZDBKEM/FUPRN
q+AG8h/KUVUN46VaZKFiaix3J2wFQnwRQfbQCT81eN2LF8Cu0tHWIbMv5yPfYEme
L7p+NaNLI62oiBSgDbfS52euJI7wwY4qkDJJNrqvU44VhErDz5Q/ewoTjviXzHVv
oB6dAaVjtjpU2dUIPK5bj10vbjlbHBVVPEYJOyBDXTGcBy4eSm56GFGLqI5G9Fev
OU7eiIcvhwMknIyCzH9Thgq/sRV2biovD+UI48xtbOrRBx2pZ+AtFo46kpXPAxrB
BYpInGtYLZzyftNyW3xql0bKeLNn7MS5boN9PvqfSY6b7d7//KGxh0PNoBUscOKf
Uq/MM6xiXJi9WKCVqPA171J4VqrMPU0LJfyQwoyLTunCO/9DUZCKc3DrKUdLAjyW
LB9GaAxoRQfmXWCbW//ZUDmzFGaeEIv85TO4dBZOYrSkIqIs6++5y2GpQ9+GqO5L
3txcpKd2Is5sFQj/7Q3AMHPlbTFQ0EF1FaJZq+0N+YHpXcKnNacUcyEZtkBiw8Vf
xj71+CqD8IrIyXuhnlzvMKkyxdZENLWJVVkX4AouKZiY6B75MLziuM8542q2ji5A
YxWu1bJu3zZnZdGgut4+HVL/GsKJqHoPiN66436Z07JnTAe/Vd+k6+ej9Acl47JS
8O/uVyrxKx8k7iXX+ojHcwZ2C72GQ2sP9kU6AvaeQFEL5+bgfd9TueMImVIuSYN+
3r+Elj2JmAZH80jPWyFTs9O4vcQn2IFlJHDYSHGbFZIY/89TXeSikO6sJgMKkmVF
aoQYlth47peNl3U/7f8zA+2Il8/Rr7vE7G3iXCQYqZeJi1J0DtfxLc0XKNYiC3mR
r4By35Nv+zyXNC5vLSGif0LaltuVaBNG/15zgKh1gr8LNKJDMA5WV+kjta5LRVt/
kwelF320frI3U+jDPmwWboAgaaQXxFbqEK/ZCWclhzwLmUpskxGfZvz7+vjgP8c4
lO29SxVKWHlmmS3yH+Snw1eaBQjjJfH15iVzPR6rW6Ura65nCIuPpM0sVIN5loVs
6howN1ZC9bR0oSabxy85hkdE6Kn7y+CW0uA/+z3B09hoT0iNIFC4Wj43ggpIA8Vf
1dvS000gIrE0ty1Vn43pRcsaWtchIZT26y/jTk7w6249wDgIK1QnrRzFhstFMHW0
WClS822q8haW9oChQJdhwqhnzUNog63w6ANzcBLURuol2Oc29t0GzZs9yyi4DgeO
cqtXWa60tEIzv6Vd2WnNe3NePGkBgIB68xJzR5g5VdSHuYa3YoyRjtO1M7cVtVFn
mkkrz7C8AZKGvTe/ms/NOLvgLrhG+cBjLf+21ItzwfoXWR+xBqjJpfxun0VFi6SN
T6dfGVYm17qYfwk7ARbsHNpi7Up/rfH2EuEfqx6CJJz8gHxFrdl/nLf0jszk+/vP
fLNO6NEQaAvHIQZTHwYLolmJfAextGxNIl0Kssgk0/DTfCnhWKxFwkspLevw7O2C
/s6I9P5s+Uut6ntudnpexFFtiKGEw3hnDC/4Xbli+051nJoIFglgaVowEgnr/ZgS
/s2dfudjCneQR8ZlBV3N+7nWr2gDebLSrzyT4TOtfDzkgZlXQuchMfDAqRDxA2sa
y208/6H3XnSxEKXOlxOqskuqsXSonofANwBlYBw/1d56gZK2mvLeGzEO6HyzBGDS
cvYHORPtipsLWaWVp3AoKUn9/FZysmDLWjLkdjXqtNOmhXz4Y7vggxXEoN3OOlS9
WbZlU1q1qD6QbmTg2S12t7hwWopwTVT/eUbyv3lYEzw1bhxt36D5YcLmLYnMYIyL
UGYDdYYcfhCAZ2iAm0F0VDPjk1hHsVpvdo8a0MuD4WWQvez1mnREatk7rFKPw3XI
6LyT3OGWu+Y9KkYCzqEI18MWtzOCNhOZ8ivIh1Myug+A9AqJqi3N8UlRdZf3j60J
U3HPuJO8T9l6W60PhB8oVJN7FKvZG+0eNc2cci/GABaj62pWURl7e0+0Tz0Pwqu4
FoWnxU2TwzHUFGu60FT5+L3H7w4OlPSpxrKRoBDtE48IFgZ5yI9UDXgTpY0EVRyb
ccGUOju+x7KgscxiE8RLwiuOWqC2zks2CCUucjzBCBQmpIXAeWLpetZFUwoih+IA
PGnoqm311jfOrBDiWQniWC6KlXJqrw0nwoT56Y8wlI+KtZy32IcBRjazLlDzMLWg
gAVta3GAOjH6qDhjkrLTNuVY1JYEQ9nPaQeDwxM8p2/jkDiaquDNFoWZoh70r/OZ
csxeTk5kvKjj44Wcr819uVkakCS0bFSH0XY8gFdcalalc7cmBgV8xLUKn87WenQN
pMG/QbQdCV61Oo7iw05/BMcp1HIsn4v63EytwFNERBrZmSWrqw3lGbW8ot9nHmfF
lETXAR4ygBWhjvvgEuEVUC1xQBZCHUN7gKxZeOlwAyY2AIM9wLi/RHbyG7AwxCO+
sQjQeMi2q/rvNm3F8/ss9tIZbvB+LS41tBe6az5oEzUUK/lxBDTXVreXKIcOXNYG
CY9aEQzQU4Hl1qmum/7N+NEN4fdiW4eN5oS2ewqLq+zGmhyIb+9nX0i6lFSLgYyn
Gm1YP+WWCPV0wCat1jprUwohKMSlW44FLqv/LTYcP0kU12Tam476/hLP+w9j4Hyl
5mYcHxaCW3GM1iGlJ8e/4RVE1VHYEAopOhgM/BdinF58dFA6SwZ5+gIGMUYHZPky
uGZhsPD+K7I11sGlirc78S8vXZ9XheREU+lkpLFlGWnvU+zviNsinlZKBdtxzmx4
Og9e4a4uT2HsIq9IQZnrwSKj919DU/lVrkZHFlBhv+OCiKy81LzkylYHWpGBoxCE
J3C8QSaiclU0BIC8aIncv7ORMmiR5dmnfFOsnkJOXSgpDwvB3OpCllF5g8mB40nj
v3CMOdF+JFFjEG1ldWdxb+LY91vOGY9T9XXMT8QALtld51qpbQQk5PmVVSykg9ei
nl9EegdwvQpf0Ff6s3Sf3xSqmyHqwPlIe9p77NFNFEh0GGU220BCSv0tvgjE3q63
BBMU2zPill0GcYz8aIw2FAnrpS9AQp96dFw1tf22jEF3UIUXyuikFmNqaIjoUGQQ
283PLrH56XAcjAyc4BQ8Wf2tnXx/8Q+FTmAoQ/H6rA3vvBHLfzD4ZnK2o5AGTMHA
qt+tMbHAp0H4F1uvuNuhvMqiTaEubKqUvoV/BKPJkf4pRWtrBqFGNnpVDvVLPGJH
+oIYsyiup+2ch9yvKD88vXWJLgH07pL5vrOB8qlTQvwA97xj+enUybY1pEygUAUM
njjJgW3FSqIvN6IUCAjLITfSYX9UhT/gtdFMJzpKYMyrPSwoPLo6ZEVF5OirQISQ
zXV2FPAVRoT/MSSiWq8R07QfTWWINPs+Brie576a8wU5EDTYg8Gkkf8jj6QQa490
iBpTj6Z5lhjpZ06+s5kK47Piz3dt+3ZqMLd0Zlv8prKAuoKU90Vz8PQUWOfeS/Md
QXI2bwCpoZG+Td9m1odoAFFsZHGcN7yG8ErtZOgaK0K7Mcv/fD6/w39jJpq8EXki
R7gi75gjdts+0bmQe5TyOumkOGeHXSy5Mc9OOEC6fYWjhkOj+C2CvwSHhERnR4x2
+8151inTbMcljjNTTVCQ/JnhV9jfM4ixHWwNl/BEETjBIJ32LWhT3zM/D5Mst4KW
6CceF9UHGQER65Vc/+Tt8Fda/kiJRchQQ3znTt71z9q39uvjdCSKA8S7tpfTk188
eGtTrQQAm3Kmab1Fr7EL0Mb8Px94seCvS/5yWwfcGUOSrFQyumRBfLpcxE2eTRzb
pupmnS+FqGlj4Yo4dE7eQVcxjttraZKUtn8FVN695PHF9vQ8zWI58K/LeExXutsH
zBO+fad1AH05Alleg41oRHF3yOdCyGLndN40cfrPQWCdzb5zAlqqa5eNRXHOJ9Hc
jwngDbAZGIrmWNhLMhdqBJc6fd4E5OggkrIBbzv1LCWIuVtlFGGyWKGGBfY7qAtU
Q8d679DyW7JMc+az4EDqmb84AJXzjsXnePao6DLKTQNHTwLvYsRotrD2NZDH117y
ipWNHCmdjiQijM1jiAByNSZqQXSKQnKLZ7DuHt00Z0oXGT5ljG3GT7ZK+olcMokg
qXv4Hs6XRF3TOa1WRWNkJr/f3K3IpfH82XS0PqwNxAQ3EMNuenrm4kcelC3ZpRsA
s1DZINXNWn3wmgD589yoiivjSpHSj911XrRyY0V3CU4f1wVD0slj/GH1t+yUKBvE
Nq5Lo95Tp2+xONtXt/ga+HmY1+MgvNQZWzQVxg9SzicNwEyRpevkfWuTnEG95QBf
epUs/ox9tPvsVuawqsVxp0VJwoysgUc/TX6WwA/6NtIziF5i8gDgpqhmnv5YY7nX
kqGDVBsQtlT/ME/RK6F6/Tfd8yBsShMSMCvxnpi6NplODUZwSOLXEoH1Xf3wano0
bl8WbN1qFY5GZPzckeV05VSrBL3hDagOGY7ZKFcm3Lp/J+sD1PVHXGbBToI8UaRv
swbYqBArMKUp5yI3tbUx0a0uOdcBiXWDm2WuCK/Ji17PQOG7ZrMReEKRTLcw4fmQ
aU6skOwqEG8jV1I9iMQJNGYnP3nccJKiMras04WmatM3ipYs/06Y8vXrSDb2JgFN
c+Du7n+lAXmSZ9obArB7vXsS6MGTypozHNmMVItBa8pkbCyArIHjnmCfHgcqTYgX
k2CiNd5n6SmUv6mxMhgUqH6dwKRhlA9PT8Em4K2Ek9TAbyZHZaebBqTRmrT0TbD2
+jepWesrBvLLdzDOpPP83rQ4zIE9Tzq98TwkL99a8e885W5pxZFo/3uVBonyUcun
+ImEjHSZDTEua9p66ecelyPlyEQmmcRBwK+4O6VCJiCHPr/nI/ZxsmrvB4g203mG
wzPa9Q+8+UQD2Tf2aDNrLizDnQpt+d+j7NEQhpSvpVoY6zPkqV1FR01wBQfzZ+js
Um8vPzmVQ/9P/8eQVSiZIaMkFVgwEu9hnkcWAxC6KWSBDiBnBaQIKoPS4vTNWLe4
I+AuoIrPk5oPXNtMYpc1yMH8J+weGn7OeeXa8VGBZ5BiDsNpMYO4kET01DiuNHEk
LVy46BgkuIy2CKkUrqjalJN3D3D8fgQbzTaVR4OHxwLpwqNbT+6tEW5Q8RzKAyU0
WJvICzToKwUTtTlRdY6KvuV3Ql/ojC7kzebuwpLH3b/5KARV13o4nT1Myg2Sechr
qAjHmj/wXR+2We2KqS2wnz6Y3d8yJagyR9sqPvqw0Ug7aXAIMOtgzFSEmoWxeuVy
6o4/1SMhFTBfwXwk5JGtDfHVhF6RqI7/LwUmipuKtOfJBIFhij2+8MDajdAokThH
qk0efW47U9LOuxqlEVMpTbfb8CvwZk/p5t+lYiwMdgUmSp8AX9xwxGIQF4Besdic
e/ZztXbRl60WO9iMp+WQEMHIIJ/dLvd8tig1IhpuC4PxoBIHqtQmvAuOKHD/qSVB
jIZaYg/2/lWGn5uO7GnCzOskh468RX4QDHZkI+vzFCkgzj5migRnVrDY5KNFwqD5
d6MXpFS++Am6/R5jMfy+bv58X9dDIZPS0uRuPjZUH+lKByBMgbbmKh51f8w6Wbae
TAy4Ggx+yLfcewnq1GFjnt6PNIt+Htuw1DErbQ4e1gzcpkeLSJGmrGK0AxVPjtIm
mCElkRkavAWFPPahlwCl5f0ZkiXGgk8m5vFcnbEXsYtvc5DoVeCPbCiylQQxtq0p
bfAw2tMKUYplwwwHmaKjrz5KwrAiNl/QP0ycO70vco9aknmwh723+1g+pxsFw/Rv
LeZSrncOc+9PdxNUun0IcBFoKN9UFQOizdKRbSZEyDc/t59fXyT+1mbAs2YOB6yv
N419UkqeGtKE2T45e54JJ5cWb4sFyAoDL8xXPPsLnI0HCuiUq0K27JuanIu6xOTw
rdCMr/McZ9CdtF9qsUrOezWq1mJZe+AVGfi2WdG5MwKR0dCAdGSn1wlo4LQj3fqg
ZWql6R+DwRYRcZ7g83fLbViquBzZMY+kMIPUJLoc3ilGpffb7bhJ21XrblIdi5E1
HUeLcpAoDcF13Wrcud1dntrbkq+qksl6OTRlqDQeM57X9ti2tB3VPo4ODCaxCz7H
kve7nazd5nk5/WgbEuO1D0es6spOEBMZqKBCGGZAVzIqmzTDQs7umrxrqg+Q7Avu
yKX13wL+9mkAJh+5GmihtR2hByzHIzXLEEcc1mw1StWxwoFIfLUqC4L1haW4oOfg
JrGoNbEyDDnHz5ZrO2zd9JlT00Ld3QL7/NldFy6n+H6wrCCEJRCWCLGtKBwdLz3c
siOxBU6VsaDQjvKzn8reI2V1lUDk7qqCrBdLHR4oA8FTQMFd+y6PeFwTCO56ss/X
udslhK6H2E0JUekVU3e2TFGqns51HRRi8xI+XpT8ooDocl2foBuV1yBzf0PgddcX
tCn9KdisQZlSObkwDe0X3kKdubkG6CQbdGXw8uDjJuPa+JuQ2hf+/ga5272qSKLo
7bodLxa6JrrKQXPywOEFSE0qM5/mPMblBqw1CbFaT2aRat65PFe0yG9FMMJGBOlE
isSpdMtdr0HC+PKCnHF858uC0vNkJ4DnXiMwALN5lKeTr5YMyvRDVegn2o3Cdslj
p0vsV//NZmrNpLYTHUT7eZ/+iYngGHti/ASzTS4e39CnajcEGOJkiuk8TcjwhpZH
WQIrR9Zneo2H7QuqASyub2MWpwlrVtJUhMTSz9RleJyzkvrdshSRXVC+QGLBxi8u
RyacmQ5i1iyL6rMnVMgNBoXCkAGCxeC7aqBnHS81sobEUHVBqOYJ+bXc6DjhuTBM
/1JI/+fwUPek3ri+PqFuawSsoHy5UWDanBHHUA9+S57TbVVoF+iCfC4aO20bBcn6
4wxSKN1HUumsCt3PtdXyMTxTVjXy5UfTjtwfkkvDzYTsREB6See/gW2t75j3aWm5
SlvcN5B4KUdRVh0kD7ZQ+KsEmx3Iy8m7HQv4sMd6Xtsr/enZ/aw+1U6UlEGj1Zrn
6zgX2m0+KII4osyDmzopc/f/6N1usC49lq1lDE95/XrxI3prfDKgpOmmhIUzedv1
wytW7m1Iy6RoVVd6gRMFKEILWIKXG5QG2/b5BnxNTpVpRRGB+kQQpLPlAr6WI1Zi
aFVRAo92jQX5eb6pxUDf8pLURNvYH5TuqdI+lH2YrhavbAPXgHBaRN+h4wXDEa+z
nnQDDNDdpW3z/Q71BopzLZ5ZQkjlfDWufTLe3o0hnI5eC0lWikgevDoXR8UlHW7R
atJIbfrPcVSacO7SPnMJBVgWYSSeFKT6c93gdAz0uH/4hP5P5mcnyM9ced4IgQO9
u1Seqx2/+HA9QveHUNXhYy2smHYGh/Vw8i/K4WBbt4tY5n2AFRBXzZsONo+3tiet
TZqhYcZzz/NupnI6hfA2L+d/KrsUBQ3fOENUDOF8Ce3K77ZH+C0+xuIEVnmOM3KL
7IYQUqRbMZ7cMIOCP4zcHX0WA1n7GMm3AmOLaYXQekPd7By1sQlAbz4X0znczO/s
pWvfEwyINerdLaO+0VrvCSF5UOo5Xv/4FP854KJOKtmRLmjwf+ZPMLRUo1nppM3Z
uqfjJzY2lQETW/8aLSo1agVzGdpw2FfyFsh/bR2AxORqQBFKcNQ1Fqjeqv2bFxSZ
OhTPBvY6C45dgsAI6u2GOg2Djs/l5GVuZeE1H6DqrG8rw6XebAVDt756Ms90quDt
aYT+N+h/ODWRhoVl5Ylag6fR39yMxOZAR1Qj7H9aM/tdlml4argwBbpSerzJGiCr
HrFaWM1wBU8Zaf6h2pTj/BYRjhGkA/hqpFwOOyAaw4zbDCCVrsvjBSqfbXGLWbx7
RnBcm5zA3e7LPPO4g0zAUXq0qYtew+AjDa/q9DhzNczUviv+Sfn5nv50/td/CVnI
pfbrHJ7nG6zpWBv4n0WpAAruTPFNizMabDu4FRC0sJac7e9I7Fxw4J2fAqnEDqt0
OqicH9oUUk5Sz6+Xqwt7y3rv4U20SUfydVX7yOyfkM9bq2vQakqPUx8jCwpwF/jk
HyCBtYkyp4Vcar8PAEeAz5SY16hHvgF7HX2YsW9rTf+1Wo38w9DTWxWCMnahTgod
cjKgzLDpJkoJHi4pBVCRUYd/UOPFb7M9eRVAcrOSNQIeV9kzhOXeTr3wQX8ZQ80Y
nJ4LaP1CEytcLeUDXoKZ3qL2+QGrZD0Q6V3h1m788P/LUC+WDzIfp+oTvqv+8bDx
C3jYD1dmasL7/Tde/yQX3hSnk57Z3t6+L7xgTAtvCZ4dMdUiy6tSN/7kSZ5/hsKw
XcbzhK0h00rZgm3ZhaZUQrKGVw8Lb1Mho5P3FyBQPsvwGXw2VfX5/ZM9wBMVeyK3
5YWGHvo9wONZS4CFfltGkTV2JFyTJAJ2NLISlsHQ0y+7tNmP9KypTGoM9WhFS+WY
qNVFswxuvrUCR/5muaX4nZ3MohZ6bMgZCDFZ0LQ4nx4EVKVQhtnp09LWl9RxJFM3
D0oYb79dozOAzl+cz/vjQDSOavup4PiWbNw+hgm1E/a0nlPQpIqdFQ3NlqDC0BMx
Uu+vjdRqfu5jIPWqN87QGqHAby+u6NJp8Q3KKt7JBplIK1tw1QALFOcmev3TmjIL
puKavAanHLGR+CQdRsza5sr9q14D+MR1w7CktdW6D0/G8KgWzXgMvjpFp27xI76S
LaV4mK7t5ywV8LpbrmjBbum+DkekHPxF+/MnjgwePbI+nidbw+7wl3s1e8RQtE9g
Gbom0wEovAV1vXMLpv4fjv5Q7mUgkWALoHdLvCynoWJJaxCYRswjaiafibuD6FKK
+mUqGMTG5C5cqxYcrnYsC6ZYhyj6KSFW54YtN4XNDnRfnzqXucFewbCCZkdGinaW
SumnX9BjaT+58a1b8x+AonImiKxJMUVq0CHB+uCs9ItjW18GAMFaYPNGh9RR5s1N
Z0JmoXmxI2lRoQieCXcV9+m34oPvNvWz1X0TqeZTA2KIuUhNjsb9FVftU+0Qql9U
mI8CLIjmygoP0n5XaA1YpdDwp+Clw7h+yo1MbSAnCd0GKqrDqzFHqeJM6/t4+pCq
CJqU+XnrS/Kakm8qbUxM1tRyXj0lar3G21quifXLVjKTdW5/gRXQArEgU6oqs/DC
rk1h7ttcdTXOHik6YEC93crX4dP+gj4/6zCG7hUaKh5RUuXIZpRpK/1piiLWQiMz
dNAlJurJWcvAwdGJp/yoUtKYQMVyENE6LPBqkkWtcWUfGy8FwKDeG0LxDZaFt1YN
ir4uMbiHos/kBHAOuBAFqpaf2WPAhBaju3CQMrBxs7b7oLaDerDGopNB0k8YwSyB
va6CmMEbqW5dfz8trZ3g67PuZHcVyyMfgPrcU3KSrghAeKsI2YTM/9NokJ9+evNX
oR8Fos5S9OmxwqEMLcBYawPpK2oJ6CpWF3fINWsYQudy2fR3SPNRAWrpVAVbXUTh
uoRPeiFe0gJ/gBHIVU67Hkfe7/D/6Pct1QlX+43/ZXtszIbD3pzzih1s+/Ndj2oh
Sb72VdLv+N6WKARK/eqjLdOSL+muRUfZsdHVksfShunzrInJPIA/o/R51aK7PVvr
SouR/HedvZ56hbRDSsdd7C2aGVMo0i5wz+BA0LZqrf6ozPqxGqvwmmuSJJl0Gg/8
h7f50P+Dwe3r0mHQNP5PGjf+IT/MHOwS9wGZehuVBqS6gE1d4qBZS0QYWiABaKju
o1DCVfFJqDIwoKakWdTovicNfFodIciKGnnvUTCIBYQ/wkTOmIaKGpK/Rah0v/hT
ZB9f6cF+Avv11UwWeI+Z1BTmG45ihfuaxVZqpzaRPW0j2ZXo3h0Mrh7Tn6SW8iVX
HpRaQoSrJ7KUS8WVNzD7gEmDhmEaX0ehgctDIpqzV7oxn8+gAQfvxYXgjQL59XOK
paFd1YJ4ExKQo899gycH4a5wFX2nFR/efq97H3oa/nPS1+l0qU4VH1atfeZKeEiC
kIngYKn8n2ljgH4xYdfRtJxHrS5huEYjyYd/udICdugEUHL0Pw0r3Q+AqOYM4AdH
Sq13vqVXeDUy5lzyRwQHEJtEtOBzMxF8GyqKSjZxQdJ37DJTGC24zUdlEJdk+tLY
LgVxOV3mYXhaboHdnYv1vWCnAbOnPzZ3adY/wISkpoGbwCCZpu3Bk9tk9rX+4+1Z
sbhyKUFyRZGCRm8UbqwI+GUmsc3TAMW95t8DLc4ypsfbFnO1zv9zrWeXoxwLHJW+
m6BDmUSZkRvUSYzZJ1vwYJ6uT5h8D7ndqo3v7PVIzKzwq+/1mhqXTPvwiEFDTEvR
AexD9ALHM8YkmWDs3q85EtVveMElz+xDwdUDZ1o2vKLt8eAMcydVq9Xv7X00PQzr
MTN+++2DX7o8WrZ1ddi7Aco1orriaCsgwxwBzWBzYCCo95+/8LBCnpypP6zik3rC
4hpdb/eFhLGFLIqyANxPg5M1oZX7L77d4Uy6ekTmU5q1eSdgmcA08gHlBSzKWLAx
WqMo6ZwP7Sf5npl6ZnFRgttqBv/Iyd7sm4EKUvKvgMK6PQkJsHT9RqzYWwj3thKx
adMFb11wG+El7JAG6o4yuDPS1rIFnMNawUH9wMjQF4NQaYQTxwMbQRO+rukuRvgq
elwyWW5XjGA2PcWYdaXGC9gA+4MqnPfaNJ07jVhXVvC8EHz8iD4UcHhsau4IR7KN
rHOHezQJJ5c/MAfWa38RyxnXz63hG3GojgYhCqPOogIWpmCVm+yAUcJtfzYngrB7
bMNspI6TrT7cXGQYpJp+Fv7yCNxYT2frHT3tDIagDX9XHSkuNmIj9m/peVvjgTkl
DHSp2NwD+IGYnaA0vqe4MKDioZ5Oev6hkxNNoTPXF7AgFKMXBREIdBHAShyt1nHo
8N0kJG61Wy5gP+GOmtAKs3ek5MFrpGP8fwvF0R4cIw0cWNKBY5rWzbeOAO4fAxQt
GidJS30QJyIo14Njds3b5DN9/oDWWqOr6dCf9mp5/C0FpqnEIrvJEHO8qTWeHVDU
bfyt1263btcTZGND4Z/9B10AhT582u60/tmNOHYhrD0hpFs9+N2TWcDL2gY9gGvn
Rs4ieaocLwfX/VpZplsOtRzsNdXAJbUKNz+mnAwOlh10lswkLyrwTVW6K74e4dSp
Zy3JcxVoDEm030QuKWVcJzpCmLs6MwBuWyFheHc2EgTafAw9xv0KjzXzgZi5uEnu
GWUwgQdEBjJTMKB1VdWoIguw1m78Ka6GbeQZerPV3bPf6xGJm/MCiwsxd0oUFoNx
XBGFns2ihB/BdU/yektWb4UxUomLfUHHl5sQGTvkRkTuBE1N8nNHb08fuA0ETTC0
78FWwPOO4NBHx3mpdovPE3N9oHSE1E40dclmbL5FpNhLUqCVHV05IrOmIGp/O0Lo
srp3Di2zg9JHMHVzYfT7BiEKxauQAWKabKZrSFy9fuOpz6nDDD0YtqmJ6EybUkam
dyS9A4BRF7BjFSGwg8HrxVfRDeW2GbUJa7fDr9qLeiKnJB/OVlx4/g4qXHFbnH3i
mO97DyjsQNCKyvAjHwJ6/UlmRPA3+TS7iZ2rDo5osehlHwAHps4f4xn201Tp2g/a
QKhPS6immR3cD4B14ai73pNNFpLEUmL4uNt3ZiDitM6KYlBtcX/82KoSPais4Tba
72PwEy5ZWw9xrjBdOxA+3gVBg4EITmyS8L2TUsyQTpaLFJMLt8Qj5QErn8sYGo59
qgDoh1Kh1r5C/ls4iXiXM+RqIPJOUGZdT9R/rf0pqz+FQgdxb6v1pmzqjo8OuvWB
AbA1TLY1vk1zNv9qJWMRJW9sLvLSfry9FMvlKLOrrI241o/2bK8aKLPUqEWlOVvh
tUcIy3/Xkt/eZei/N1kRIb11UJv2jaOrX+WYu47zaxNrlDubd5HhWiQGrXcCoRMW
1YkbdVbMOvdy0OlxI6uf/XuI4nsDG5Hu625g3SWogtKKZLbP9Kou+WwIgPHG6bII
fb633zmab+jR5fvT5ls0VdIIkjuR1o5od/RX4G+0kkusGLU8+b6gZSFeB1R8HcMi
8cfHjNCCpfidRmfRrYNmZgKpZBxcfvq0UmrI8V00aW0/cXaaPud0XixJS4k78Uhd
nHEA19iI931JUVPTvYfodAC0MJkMI1H7ttjZ1UmBqHzKzn8cBGrEZn3GDz9hDeBN
jAITPQOriozdbw70O8eTAXgXUoNK5Odygk9iZ7lqoh2W9BcP1L3urmivE+HfkiCz
hJgWB6fTwGLM/jb6q5MQm8KdtIjkNvqSVgzwFmlHQkmkO22egUa5kOYB7Uw+xp04
GhcOW5locaJMEYnz4N4XgzN9H0uaoSMLyz1J1EJKFdyrzPAN6cxpJrOfQUkGoUDV
c2LJezZN009PTCld4GL1pHXtWDWHOq7S7fuDnDpI2AqysztIDnd/FKFj/dVIxvjI
nngOe/931FzTYSgtvY7kctydCVwbsHZwrFDZHVnniM+0am1AxUVcLOGhsViT8ihN
pJWqxOYoqNeJWay6ZgVfNsSdyZPvbZ//R6C/NKbByvh+mefEMkiJFesAbg8D2Hhr
pCTEK+A45NAGFT6A5WIjVtgsp0bDTOERXJW8OFmzgaH1BeUMEPUEbI4IDymeic5I
TKOhpdRtQvDDgyRywDeYYmLC2zksai+2wioF/llCClVMJ2l0dULoIahAI8lsZpp8
Gzm4FXPdE9otqdnGyW8NqTYG88Ou0i3tcn9jGwgHzWuVp1W7vM57x5ju+aAfbQrV
ENbt5c4dIr+iOyxFIe9Y/oovwX66sR+LtBD7cDgDKy7uZ7dVLLL7wvs0n/DilSMh
lEjTiHGuXFdf3RExkJKU/Q7o3y/Jkd2r+tx2xjKdVaJhZpeoe9Ocbeyc8Sk27GJe
Vnj7NBAaWXcrSvrTnOuTP0ewAVbToTvLez+bjgsidXo7giMRj31z3zW4bdX68TA/
kZP9XXSHtSz8NFq7Imz5+WsdY4NuY/hSjGeeJxaGVhPI0zCwAHjcGCw0/yJL6qki
cBbVhHT8BoB67ZxFZV4FQGXbILC6J8Qo6CKOeFtX5U80MPhL9ku2pS69lpwNnTdp
ev0O2MOlmUCUs5Hpxk1vcmJXeLmWA3COYF8er0ptt5UGTdluNr8O8MGxNOTllKYk
2Ry9CkjsWIXReRgie5Qn8evwXpot6SgyDO1zBFB8iIseqq6a2b/PSWL8D2iOEwFZ
wN7GnLJSFXLr9I2wctmfKH9iu8Z70OYyzEhgsREMMvERljh51vhWCnxw8xDr3E9X
DT9E5PtjoPrMUNb1lHkwZo6kXr+TE12auerKJIFwSl8S71E4zyCPmNobfYvIcG/E
iAKJFyZRmAKgg0r1AcKf6CHVf4//tIsoWX7MTTCTqAnGyQs/naCcvDnD8nC71a7n
qZwUXHAL8chqkCBhrk0zM+n4ORkloapT8JvB8wCAk6FAK6pdN1ENc7HsFMrcmVtG
jjYRI78UXXP7Qqtlq1mYNBwlmvedB0VF7nwvLf8EZ8bXoddjb5fs2RR+/LRW06CU
NOUxpqRpEIGkgmsBgbgtFLVMpavhNrKZKnC83+m9NjFG8IyF7l4MSyMjuVlmmJRq
0ays8A+XZlCk6h+Z1S1+PMQYYuyWHinz/mGdHZcjjkXC9nnP1oahr0sZnj0IcjZ4
/Iu4n4XO3wLGGHotYE3g8xJW2Q9TRcep4ZKnbFs+pUuDvgnirRwCul1pdyYQTSG4
o3D6JEg+WoAeWYnd0x8EHsGrYfY23Qq3SmS8ByALAzbsFDqKpxiQmdzji5NyxPDo
TSPF0VLf8gG9iYsFJLtLFs928aI/ThC/BrHuAkbunzqm8P7vbdBTxHPrFgbb8306
DPX2It1gdb8VSpU+fXU56A8PO9Ee+5Nj+Qi+YaSsRLzcxdhOaYkWjb6hl3bmOS+L
hKUsMRRklePe+8vsmBZKVFRhMv9rywB2OdmBRHAnLggIngGWE/mDyMepBTVNHTlg
zbePW8DnsWy+6bLovpMPPxGJFCvTgEPmeM9MZWRgM6J+k6Syh9RZWiSduWBQpFJs
vgOAKZnbZiiJL0og9lLMfvHChwwU5l/MvdtlZowGMbsaG38EwuKlVy1S1Vk6QsOm
PtOEer4s0t/PeAlZ5xfyBB0irk8I7Ygl/cmfoCVHHx/wvlzSXGn25zXXk1ygrQvb
QC6nU2+41gxXo9Hd+OkM7Zx5Xfc9u5HSWDnt8viUj5U2oNh5SrNHshlk9kik3lIS
AGXnEKhN9u347+WsfhyRQ/ygMc/0rcm3RWqqhudu/XwVnw014GyAynzTU0F6WNOA
zoTDcYxjnvNAPoMzWbkapNI3r5Mf0UIfEF33ge2XW3vt0z2Xu5tmMlwgM4iq00AQ
dJpEXOXNgfo2m4/pC0CoDonWTWHX7gkiKWCVY6Bz0EVrd7PulaLvAe6p93FUbZRN
VEgJKL/pcdss4i/o2WxkpEZ0US8miHLhiguN1FeZLw1+ziMbLjuw85KrGrbk/v1U
CmYakN9cPocGmK+M3GphirBVrjzu8qZ4hQmSvg8tpyHS+T5n5l4KgKJJ8HDC3HgA
ZZ2Ubo2R+RBn+ouKpEpy/cKBOsuJNXnxTfHunosTqSjlyCg4qT7rD7c90XNtb/jD
Mr5AjwcB00xYK1SB45mof7maWRRaQB/TYpHdeyif+fCbcg2x7I4TZb7ZrHzsMe7B
0E2x5Co5nV3QrSbM1zcTDQR09v4BfZjLOfR3gDWqf2Sa1ACq9QSDkR85FCJUQUxn
vUu9XkfJojEsTnX+jzhYejA3FI7toAUO3KEoPrq8wczlBAwMHD/L7P9JR+7eNoIS
xW0e89qpySvl+SfHdMEuE5i2Whv1qDxHhThyJXsP4EErPBhOJqoO+D3hST3leZkW
OQqRqjLvj4MuU27B5TM3QQETR+kF/P86FB+/+Xo5sHvu+1igiOVJZemThirnCxcs
lAcSIXj/9Ca0tfxYL/YKaKsSdRvHyrmGDHCqHu1BZJ+g5lMadSEvDhhU/BobJwoO
aBQntnkPfF/+PDikteh0SZ1h2UYtH148N3hrpBjQ5PXUL8P2oyrEKx9ZQI2G3ygG
zUDAf1vDGl9sVX4wubeYb0iFd55emO9Ey5rRROzF/eVIE15QLccSSKb6B8goMnfD
mgN9GrFqoxX14DamwTlOr6pKyRG8/+unOmi5HKbr14tFtRFxxTa1eySaILmGcShN
AcTSuF9mPAsd6ydAxsSK69tqaOuUMRwnmfNxLfwWVRoVefLYfh+tDAu602qIknJh
phSBjsg/auhZK8Gst5L/xLG/SnDo4k0O7h/woWF5YnNpHcu38qfpbUIKybhC96Uj
iocZtuT3DVEQofcLJCP3tz0gdA0oGtmzG6kud9Fz/fG8T2kfTGYGk+7yCxAnBUTp
oQ+eORV+UpBEb5NVO63PAURtkAXsoTn1wNbXxO5YLLTK6eyKHWKue5gqr7+PMduc
yzDI0NjGQWWRvBsuT+OcD3sJ/bAPMa5gsVbyPZwO4xnfUI17r+Eb+xJqlJpAjF3w
i+AEHa5BGy/4IpWNizd3V7uccyS3jn0mSkNz6KUjzCBOM6VTVZj59SkETWtod7PS
Uljpxpfg+m7XL36CGAkM08w+5FtpDC3Wxw4DZxClY58X9CpIKvBWA3CScH/lCqMv
m1z+p6oirVnpXq1tY7AReSahzUzg7LjeRG6nN5iI54IEuqW07lbkW5z+bsB2z5SC
r/PE2qKosZ+Y6fxYsvbuWx8OEIYNNH+qb23HI27z5CID6e+YEamEeDwm0bkfRiuG
aB4OKHeDq36XzUuN2e5a5VBEZlXC4Ep/lQmhWaQiJJL0jj9R2dMkM93pkE+05Yi7
weuT1PUPOTav4FER7NbF5zMDOt3Na0Go1JD0kvrfgrqu4jZMyfBo70fzg+Y9pZL/
Ftlq6f/hyFX6cQi+0qU5nov3jvylxalq8TsQoUh2OX0sO4zTP3l4P3qc8j+QGMj9
KmtUb82l8+fRt8vjihGYBd+xqfzIfla0DvceUW+6VYlkaqO4ID3Y1hzKQ3reOLTu
ZBp0+NFPyH5DB85GPi2jHZ/VsyDN1dtxZbtqWQo4kAcUax9DfxQxgNgBUK7N3/n8
EylAbvOozQ5LkGQBXKspIbx/64mX0nPn2O1rjJxrOeC935TsDoXIKcpJzYs5PgY2
zM7CQOk2d+UtawQvc+bpJyRSfjr4lV/VGzQ6HFbh2b8s1ofd7FUXoFZjshQxRhYl
16EiQQKxcOiiOCkn6JaPn2dztJzyMn+9wT4gXpd89VMWiMPAFKoB9U5leUr2mfUQ
MvUFdBedRPXzp8aZKBMe/Qk/s9+8FGvDwZsZEcNjyTdVGaGhQhfmXiLuZtpnFdq3
chaCmLRKnqy0rGgqrFHi4o/SlNg3JZ7CyljVLNg8fsURRbhgWQMmKMNRlsSjAuVl
wLb+EiS4yMwXdg5+K6wpGgXWLzrNi++punFkCyHdAkk47dKqrrJtNu4WOLGcMR0p
X9INozMYXNzoI4ocFgpmM7ZaN+vSfn5KVsVz462ZAewdEuhPBWlLAaqEWLU/RFoJ
2j2/HRGHjl5K
=9HfX
-----END PGP MESSAGE-----
EOF


---

### Step 5.5 Create your secure build server custom configuration

Anybody who uses the secure build feature of HPVS will use the same Secure Build Server image, but that image is deployed to that person's individual HPVS instance, and that individual instance is uniquely customized through a configuration file. This ensures that your Secure Build Server can only build the application you specify, and push it only to the registry that you specify in the configuration file which is created by the below code cell.  Don't run it yet!  You have to tailor several settings in the code cell below before you run it:

1. For *both*:

    a. _DOCKER_REPO_

    b. _DOCKER_USER_

 replace `YOUR-DOCKER-HUB-USERID` with your Docker Hub account userid. You either already had an account or you created one in *Step 3.1* of this lab.


2. For _DOCKER_PASSWORD_:

 replace `YOUR-DOCKER-ACCESS-TOKEN` with the value of the Docker Access Token that you created in *Step 4.2* of this lab.


3. For the value of the _ACCESS_TOKEN_ key, which is within the _ARG_ key at the bottom, replace `YOUR-GITHUB-ACCESS-TOKEN` with the value of the GitHub personal access token that you created in _Step 2.5_.

Leave all of the other values unchanged.

Once you have made these substitutions in the below code cell, run the code cell.

**Note:** This code cell also does not produce any output within the Jupyter Notebook for the same reason as the previous code cell.

In [ ]:
%%bash
cat <<EOF > sbs-config.json
{
    "HOSTNAME": "sbs.example.com",
    "RUNTIME_TYPE": "classic",
    "CICD_PUBLIC_IP": "",
    "CICD_PORT": "443",
    "IMAGE_TAG": "1.3.0.9",
    "CONTAINER_NAME": "SBContainer",
    "GITHUB_KEY_FILE": "~/.ssh/id_rsa",
    "GITHUB_URL": "git@github.com:IBM/secure-bitcoin-wallet.git",
    "GITHUB_BRANCH": "master",
    "IMAGE_TAG_PREFIX": "s390x-v1",
    "REPO_ID": "sbs",
    "DOCKER_REPO": "YOUR-DOCKER-HUB-USERID/secure-bitcoin-wallet",
    "DOCKER_USER": "YOUR-DOCKER-HUB-USERID",
    "DOCKER_PASSWORD": "YOUR-DOCKER-ACCESS-TOKEN",
    "DOCKER_CONTENT_TRUST_BASE": "False",
    "DOCKER_CONTENT_TRUST_BASE_SERVER": "",
    "ENV_WHITELIST":  ["ZHSM", "APIKEY"],
    "ARG": {
      "ACCESS_TOKEN": "YOUR-GITHUB-ACCESS-TOKEN"
    }
  }
EOF


---

### Step 5.6 Verify your custom configuration file

You can run the code cell below to display the contents of the file you just created to ensure it looks good.  If you made a typo, you can probably fix it by going back to the above code cell and running it again. 

The code cell below pipes the output of the _cat_ command to a utility called _jq_ (for _JSON query_) which expects well-formed JSON (JavaScript Object Notation) data. If you made a mistake while updating the file that causes a JSON syntax error, the _jq_ utility will detect it and print an error message.  

If you put in a wrong value that doesn't cause a JSON syntax error, _jq_ will not catch that, so inspect the output of the code cell carefully.  All variables that you are required to change started with _YOUR_, so it isn't a bad idea to look for the word YOUR in the output—you shouldn't find it!

In [ ]:
! cat sbs-config.json | jq .

---

### Step 5.7 Generate client certification authority and client certificate

Secure communication between you and your secure build server is achieved by using mutual Transport Layer Security (TLS). TLS is the same security protocol that protects most Web traffic today, but most Web interactions only require the server to be authenticated.  Mutual TLS refers to the situation where not only does the client want to ensure that the server is legitimate, but the server also wants to authenticate the identity of the client.  Your Web browser is constantly receiving server certificates and verifying them while you surf the web. Client certificates are what are used when the server wants to verify the client.  

Create the client certificate and client certification authority (CA) by running the code cell below. You should receive two lines of output, with the second line being `INFO:root:client_certificate: generating client CA and certificate`

In [ ]:
! ./build.py create-client-cert --env sbs-config.json

---

### Step 5.8 Generate server certificate and server key

Create the server certificate and server key by running the code cell below. You should receive four lines of output, with the fourth line being `INFO:root:server_certificate: Successfully generated server certificate`

In [ ]:
! ./build.py create-server-cert --env sbs-config.json

---

### Step 5.9 Display your environment which shows your client certificate and CA and your server certificate and key

The next code cell will display the client certificate, the client certification authority, the server certificate and the server key, all in base64-encoding.

**Note:** Keep the output from the below code cell.  You will be copying most of its output into your clipboard in a subsequent step.

If you already know about _base64-encoding_, go ahead and run the code cell.  If you don't, read the next paragraph if you're curious about what base64-encoding is.  

Base64-encoding is a means of encoding any data, including binary data, into human-readable characters.  It takes groups of 6 bits, converts these six bits into a number from 0 to 63  (2 to the 6th power is 64), and replaces these 6 bits with one of 64 human-readable characters.  Which 64 human-readable characters were chosen?  Lowercase alphabetic characters a-z for 26, uppercase alphabetic characters A-Z for 26 more, the digits 0-9 for 10 more, which gives us 62.  Then `+` and `/` were chosen to make 64!  It should be noted that base64-encoding should not be confused with encryption.  It is easy to convert a base64-encoded string back to its original format.


In [ ]:
! ./build.py instance-env --env sbs-config.json

---

### Step 5.10 List your current HPVS instances

Run the following code cell to list your current HPVS instances. If you are new to HPVS, you won't have any.

This lab uses the free version of HPVS instances, and IBM Cloud users are limited to two free instances at a time, and the lab requires two instances. If you are an experienced HPVS user and you have paid HPVS instances, they won't be affected by this lab.  If you are currently using one free HPVS instance and don't want to delete it, talk to your instructor who can help you work around this situation. If you are currently using both free HPVS instances and can't delete at least one of them, then unfortunately this lab will not work unless you are willing to incur the costs of creating chargeable HPVS instances for this lab.

In [ ]:
! ibmcloud hpvs instances

---

### Step 5.11  Create your Secure Build Server with the _ibmcloud hpvs_ CLI

The code cell below has the first part of the `ibmcloud hpvs instance-create` command filled in properly.

**You need to make two changes.**

**First**, you need to change _YOUR-LOCATION_ to one of the locations as explained in the following paragraphs.

If you logged in to _us-east_, then choose one of these three locations: `wdc04` or `wdc06` or `wdc07`

If you logged in to _us-south_, then choose one of these three locations: `dal10` or `dal12` or `dal13`

If you logged in to _au-syd_, then choose one of these three locations: `syd01` or `syd04` or `syd05`

If you logged in to _eu-de_, then choose one of these three locations: `fra02` or `fra04` or `fra05`

**Second**, you need to add arguments for the `CLIENT_CRT`, `CLIENT_CA`, `SERVER_CRT` and `SERVER_KEY` variables.  All you have to do is copy them from the output from two code cells above, from _Step 5.9_.  Copy the four variables by selecting the output starting from _-e CLIENT_CRT_ and extending to the end of the output. In other words, you are copying almost all of the output from _Step 5.9_ except for the few lines of output that precede the start of the first *-e CLIENT_...* variable that is listed.

I have left a blank line in the code cell below in which you can paste these four excruciatingly long environment variables. Click in the code cell so that your cursor is at the beginning of the blank line 2 of the code cell, and then paste in the variables at that spot.

**Tip:** When trying to copy this long string in the clipboard, when the entire string isn't visible on the screen at once, I have had smoother scrolling when starting from the bottom and then sweeping up to the top to select the text.  Your experience may differ.

Once you have tailored the code cell per these instructions, run it.  This command may take several seconds to run.

In [ ]:
! ibmcloud hpvs instance-create SBContainer free YOUR-LOCATION --hostname sbs.example.com --rd-path "secure_build.asc" -i 1.3.0.9 \

# copy and paste, in the blank line above me, the super long -e CLIENT_CRT=...  -e CLIENT_CA=...  -e SERVER_CRT=...  -e SERVER_KEY=...  from the output of the command \ 
# a couple cells above.  you can leave me here because I'm just a comment.

---

### Step 5.12 Check the status of your Secure Build Server HPVS instance

It will take several minutes for your instance to be built. The command output from the previous code cell in *Step 5.11* conveniently lists the _ibmcloud hpvs_ command you can use to display the status. Copy the entire `ibmcloud hpvs instance crn:...` command above, which has your unique instance id (shown in the output of the above command, it starts with _crn:_) into the code cell below and run it periodically (about once a minute) until the instance is successfully created. 

**Tip:** Your instance is successfully created when the value of the _Cloud state_ field of the output is _active_. For the first several minutes you will see that the _Cloud state_ is _provisioning_ and that most of the values listed below _Cloud state_ are _not available_.  At a certain point, you will see that those values that were _not available_ have been populated, but you may see that the _Cloud state_ remains as _provisioning_ for a little while. Ensure that you wait for the _Cloud state_ to change to _active_ before proceeding with the lab.

In [ ]:
! 
# copy and paste above the entire 'ibmcloud hpvs instance crn:.....' commmand from the \
# output from the previous command, and run it periodically (every minute or so) until 'Cloud state' is active.

---

## Create your sample Secure Bitcoin Wallet Application BYOI image ##

### Step 6.1 View your custom configuration file again

The code cell below will show the current state of your secure build configuration file.  You have already looked at this file from an earlier code cell, but we're repeating it here so that you can observe the key thing I want you to note from the output—that the IP address value of the _CICD_PUBLIC_IP_ key is blank.

In [ ]:
! cat sbs-config.json | jq .

---

### Step 6.2 Add the public IP address of your secure build server to your custom configuration file

Now that your secure build server instance is running, make a note or copy the IP address shown in the *Public IP address* field from the command output from *Step 5.12*. The code cell below uses an awkward-looking, yet simple _sed_ command to add the IP address in the proper spot.  

**Editing the below code cell is required:** Replace the _xx.xx.xx.xx_ string below with the Public IP address of your secure build container, then run the code cell. Make sure you leave the double-quotes surrounding _xx.xx.xx.xx_ in place. You should not see any output from this code cell:

In [ ]:
! sed -i -e 's/\"CICD_PUBLIC_IP\"\: \"\"/\"CICD_PUBLIC_IP\"\: \"xx.xx.xx.xx\"/' sbs-config.json

---

### Step 6.3 Verify that you added your public IP address correctly

Run the code cell below to repeat the command that displays your configuration file. Ensure that your public IP address was added properly.  Instead of _"CICD_PUBLIC_IP": "",_ the line should look like _"CICD_PUBLIC_IP": "xx.xx.xx.xx",_ but with your Secure Build Server instance's public IP address instead of _xx.xx.xx.xx_.

In [ ]:
! cat sbs-config.json | jq .

---

### Step 6.4 Update your /etc/hosts files

You need to update the _/etc/hosts_ file in your lab workspace in order to map your secure build server's Public IP to the hostname of `sbs.example.com`. You will want to add a line that looks like this, substituting your instance's Public IP address for `xx.xx.xx.xx` in the below example:

`xx.xx.xx.xx    sbs.example.com`

Run the following command in your terminal tab to get to an editor and update your /etc/hosts file. Ask an instructor for help if necessary if you are not familiar with the _vi_ editor:

`sudo vi /etc/hosts`



---

### Step 6.5 Check the status of your Secure Build Server

Run the following command to check the status of your Secure Build Server instance. The value of the _status_ field in the output should be empty, i.e., `"status": ""`:

In [ ]:
! ./build.py status --env sbs-config.json

---

### Step 6.6 Initialize the configuration for your Secure Build Server instance

Run the following code cell to initialize the configuration for your Secure Build Server instance. The status of the output should be `OK`:

In [ ]:
! ./build.py init --env sbs-config.json

---

### Step 6.7 Start the build of your sample secure bitcoin wallet application image

The code cell below will start the build of your application image. The command returns quickly, as soon as it submits the request to start the build.  Your status message from the command output should say `OK: async build started`, which indicates that the build of your image has been started up on your Secure Build Server container. The command does not wait for the build to complete. The build will take about twenty minutes. Try to check out the next two steps before you take a break.

Run the below code cell to start the your build:

In [ ]:
! ./build.py build --env sbs-config.json

---

### Step 6.8 Check the status of your application image build

Check the status of your build by running the code cell below. You can check it periodically—your build should take about twenty minutes to complete. You can run this code cell periodically, as well as the command in the subsequent code cell (_Step 6.9_) which displays the log messages from the build.  For now, check the status by running the code cell below.  Your build has completed successfully once the output of the code cell has the value of *success* in the *status* field:

In [ ]:
! ./build.py status --env sbs-config.json

In the above code cell, until the build completes, you may see several different statuses, including:

_cleaned up_   This status can appear at the beginning of the build but you may not see it as the status quickly changes to _github cloned_.

_github cloned_  This status appears most of the time as this status remains until the image build is complete, and this takes the largest amount of the entire build time.

_image built_  Once the image is built, but before the push of the image into your Docker Hub account is complete, you may see this status.

_image pushed_ Once the image has been pushed to your Docker Hub account, you may see this status, but usually the build completes very soon afterwards, so you may not see this status, as it should hopefully quickly change to _success_ status.

_success_ Once the secure build process has completed successfully, you will see this status. Once you see this status, you are free to move on (Step 6.10). Remember, you can run the command to display the build log periodically (Step 6.9) while you are waiting for this _success_ status. 

---

### Step 6.9 View the log from your application image build

Your secure image build will produce a lot of output in its log. 

You can check the log of your build by running the below code cell. 

**Jupyter Notebook tip 1:** In some environments this output may be placed in a scrollable window, but in other environments all of the log output may be placed into the notebook, which makes navigating to the rest of the Notebook more difficult.  If this happens to you, you can enable scrolling by clicking in the output, then right-click and choose _Enable Scrolling for Outputs_. This will place the output into a scrollable window. 

**Jupyter Notebook tip 2:** Additionally, you can clear the output of this command from your Jupyter Notebook by clicking within the output, then right-click and choose _Clear Outputs_.  (The build logs are retained on the Secure Build Server and can be retrieved at any time).

**Jupyter Notebook tip 3:** Running a code cell clears the output from any previous execution of the code cell, so if you want to view the log more than once, you don't need to clear the output as described in the tip given in the preceding paragraph.

In [ ]:
! ./build.py log --log build --env sbs-config.json

---

### Step 6.10 Download the state image of your Secure Build Server

Run the code cell below to download the state image, which is a backup of the state of your Secure Build Server instance. This can be used to recreate your Secure Build Server if necessary. This backup also contains the logs of any builds you run on your Secure Build Server.

In [ ]:
! ./build.py get-state-image --env sbs-config.json

---

### Step 6.11 Verify the manifest file from your build

The Secure Build Server creates a signed manifest file for each successful build. The manifest file is useful for audit purposes. You can verify the source and integrity of the build and the built image or you can pass the manifest file to an auditor to do so. Run the below code cell to download and verify the manifest file:

In [ ]:
! ./build.py get-manifest --env sbs-config.json  --verify-manifest

The above command downloads the manifest file and verifies the digital signature on it.  In the last line of output from the above code cell you should see *verify=OK* which indicates that the digital signature is valid and is proof that the manifest file has not been tampered with.

---

## Create HPVS instance using the BYOI image that you just built

### Step 7.1 Download the encrypted registration file for the image that you just built

In order to create an HPVS instance using the application image you just built, you must download an encrypted registration file for your application image that was created by the build. This registration file can only be decrypted, and thus used, by your application container image.

**Note:** This command won't work if run in a code cell because of the need to type in a passphrase. That is why you have to run it in the _Terminal_ tab.

Copy the command below and switch to your _Terminal_ tab and run the command, and enter a passphrase of your choosing when prompted.

`cd /home/jovyan/labdir/secure-build-cli && ./build.py get-config-json --env sbs-config.json --key-id secure-build-7f649cf-1`

You will be promted to enter a passphrase three times—the first two times to set the passphrase on a new key, and then this new key is used to sign the registration file, so you are asked to enter the passphrase you just set. For this lab, you only need to remember it for the duration of this command—it's not used in any subsequent commands in the lab. If you enter a simple passphrase, you may be warned about this, but it's okay to accept it anyway for this lab, if you promise not to be so lackadaisical in the real world!

Note the value chosen for the *--key-id* argument:  *secure-build-7f649cf-1*. 
If you look at your command output a few code cells above, from _Step 6.8_, where you checked on your build status, you can see _7f649cf_ at the end of the value of the *image_tag* key, i.e.,  `"image_tag" : "s390x-v1-7f649cf"`. 
This value _7f649cf_ was set by the secure build server. It is not unreasonable for you to wonder why the secure build server chose this particular value. It would be unreasonable for me to fail to provide an answer now that I've piqued your curiosity. That value is the beginning of the hash of the most recent commit of the branch of the GitHub repository that you specified in the secure build server configuration file that you created in *Step 5.5*. Check it out for yourself at [https://github.com/IBM/secure-bitcoin-wallet](https://github.com/IBM/secure-bitcoin-wallet). The secure build server appends this to _s390x-v1-_ in order to form a tag name for the image it has built. 

The *--key-id* value can be anything, but putting this hash value as part of the name helps to keep from confusing different keys if you have multiple keys.



---

### Step 7.2 Launch your BYOI application as an HPVS instance

Make the change described here to the below code cell and then run the code cell to create your HPVS instance using your BYOI image that you just built.  The value for the `-i` argument must be the value of the *image_tag* key in the output of *Step 6.8* which checked your build status.

**Tailoring the below code cell before your run it:** You need to make two changes:

1. You need to change _YOUR-LOCATION_ to one of the locations as explained in the following paragraphs.

    If you logged in to _us-east_, then choose one of these three locations: `wdc04` or `wdc06` or `wdc07`

    If you logged in to _us-south_, then choose one of these three locations: `dal10` or `dal12` or `dal13`

    If you logged in to _au-syd_, then choose one of these three locations: `syd01` or `syd04` or `syd05`

    If you logged in to _eu-de_, then choose one of these three locations: `fra02` or `fra04` or `fra05`
    

2. Ask your instructor for the APIKEY which allows access to the instructor-provided Hyper Protect Crypto Services instance, and replace _YOUR-INSTRUCTOR-PROVIDED-APIKEY_ with the value that the instructor gives you.

In [ ]:
! ibmcloud hpvs instance-create securewallet free YOUR-LOCATION --rd-path sbs.enc -i s390x-v1-7f649cf \
-e "ZHSM"="ep11.us-south.hs-crypto.cloud.ibm.com:8367" \
-e "APIKEY"="YOUR-INSTRUCTOR-PROVIDED-APIKEY"

---

### Step 7.3 Check the status of your BYOI application instance

It will take several minutes for your instance to be built. The command output from the previous code cell in *Step 7.2* conveniently lists the _ibmcloud hpvs_ command you can use to display the status. Copy the entire `ibmcloud hpvs instance crn:...` command above, which has your unique instance id (shown in the output of the above command, it starts with _crn:_), into the code cell below and run it periodically (about once a minute) until the instance is successfully created. 

**Tip:** Your instance is successfully created when the value of the _Cloud state_ field of the output is _active_. For the first several minutes you will see that the _Cloud state_ is _provisioning_ and that most of the values listed below _Cloud state_ are _not available_.  At a certain point, you will see that those values that were _not available_ have been populated, but you may see that the _Cloud state_ remains as _provisioning_ for a little while. Ensure that you wait for the _Cloud state_ to change to _active_ before proceeding with the lab.

In [ ]:
! 
# copy and paste above the entire 'ibmcloud hpvs instance crn:.....' commmand from the \
# output from the previous command, and run it periodically (every minute or so) until it is running.

---

### Step 7.4 Edit this markdown cell to create a clickable link to get to your sample application

Once your instance is active, note the value of the _Public IP address_ field from the output from _Step 7.3_.  

Double-click anywhere within this paragraph to go into _edit_ mode of this markdown cell.  Then, in the final line of this markdown cell, change `xx.xx.xx.xx` to your public IP address.  The actual URL is on the right side of the line (between parentheses). The text of the link is on the left side of the line in between square brackets.  You may as well change both occurrences of `xx.xx.xx.xx` for clarity, but the one that matters is the link on the right side.   (If you don't see both sides you haven't succesfully got into edit mode—keep trying or ask an instructor for help).  After you have made the change, run this markdown cell by clicking the *Run* icon in the Jupyter Notebook menu above. This will render the markdown and present you with the properly formatted link. Click the link and you will bring up your sample secure bitcoin wallet application.

[Change the xx.xx.xx.xx to your public IP and then click here](https://xx.xx.xx.xx/electrum)

---

## Running the Secure Bitcoin Wallet Application in your BYOI HPVS instance

### Overview

This application is a secure [Electrum](https://electrum.org/#home) Bitcoin wallet found at [this GitHub repository](https://github.com/IBM/secure-bitcoin-wallet). You can use it to store the identities necessary to transact on the Bitcoin public blockchain network. While this version of the wallet is just a proof of concept, running this type of wallet on Hyper Protect enables you to protect your digital assets from bad actors such as malicious server administrators.

*This Electrum Bitcoin wallet functions regularly, while inheriting the security benefits of the Hyper Protect Virtual Server platform. This ensures that a legitimate version of the application is running (Secure Image Build process) and that administrators cannot compromise the application (Hyper Protect Virtual Server deployment).*

Here is an overview diagram of the _sample secure bitcoin wallet application_ that you built in this lab: 

![Secure Bitcoin Overview](images/sampleapparch.png)

The application, by default, uses software cryptography in order to generate keys, but it can also be configured to use _Hyper Protect Crypto Services_ in order to generate keys more securely, which is what you have done earlier in this lab when you specified the _ZHSM_ and the _APIKEY_ environment variables in _Step 7.2_.


### Step 8.1 Account Registration

Registering a fake account for lab purposes.

1. Choose `Register` since you won't have an existing account on the newly deployed server.

    ![Secure Bitcoin Wallet Welcome](Bitcoin_Wallet_Images/Bitcoin_Wallet_Homepage.png)

2. Enter the registration details you desire.

    ![User Registration](Bitcoin_Wallet_Images/Wallet_register_info.png)


*Do not worry about the information you enter on the registration page (shown below) as it won't map to anything critical to the real world.  You just need your password to log back in if you log out of the application.*

### Step 8.2 Wallet initialization

It's time to set up your Secure Electrum Bitcoin wallet itself!

1. Bring up the wallet screen

    ![Wallet Start](Bitcoin_Wallet_Images/Click_Wallet_Start.png)

2. Enter in a `password` for your wallet and click `Create Wallet`

    ![Create Wallet](Bitcoin_Wallet_Images/Create_Wallet.png)

3. Load your newly created wallet to enable it

    ![Load Wallet](Bitcoin_Wallet_Images/Load_Wallet.png)

4. Reload the page

    ![Initialized Wallet](Bitcoin_Wallet_Images/Reload_new_wallet.png)

5. Your wallet is ready for use in all of its glory

    ![Locked and Loaded Wallet](Bitcoin_Wallet_Images/Loaded_Wallet_Time.png)

### Step 8.3 Receiving Bitcoins

A wallet without currency is blasphemy, so it's time to fill it on up with fake bitcoins.

1. Go to the `Receive` section of the wallet and get the bitcoin wallet address.  

    ![Wallet receiving bitcoin](Bitcoin_Wallet_Images/Receiving_bitcoin.png)

    *This is the address whose alloted bitcoins you lay claim to.*

2. Open [this bitcoin test faucet site](https://bitcoinfaucet.uo1.net/send.php) [click on the link] and use it to transfer bitcoin value to the `Receiving address` in your wallet **from step 1 above**.

    ![Transfer Bitcoins](Bitcoin_Wallet_Images/Send_bitcoins_to_wallet.png)

    *Instead of transacting on the "real" bitcoin blockchain network, we are using a [testnet](https://medium.com/myetherwallet/understanding-blockchain-changes-testnets-and-mainnets-c2171a8e835f). This means that the network participants agree that the assets on the network have no value. Testnets are used for testing changes to the network, testing applications and introducing new users to the technical aspects of network. There are testnets for many of the public blockchain networks including of course bitcoin and ethereum. Test faucets (like the one we are using) act as distributors of tokens to other users on the testnet.*

3. Switch back to your wallet tab and go to the `History` section and then reload the page.

    ![Check History](Bitcoin_Wallet_Images/Check_History_Receive_Transaction.png)

4. You should now see your wallet address as the recipient of a `pending transaction` from the bitcoin testnet faucet.

    ![First Transaction History](Bitcoin_Wallet_Images/Transaction_History_no_confirmations.png)

5. After clicking for more information you get a detailed transaction overview

    ![Receive Transaction 0 Confirmations](Bitcoin_Wallet_Images/0_confirmations_block_details.png)

    *The number of [confirmations](https://en.bitcoin.it/wiki/Confirmation) a block receives is the number of blocks deep the block that has included the blockchain transaction is. In other words, `1 confirmation` means that the current head block includes the transaction. `2 confirmations` means that the current head block's parent block contains the transaction and so on. The deeper the transaction is (more confirmations it has) the more mining power it would take to "reverse history" and undo the transaction.*

    Since there are `0 confirmations` in the picture above, it means that the transaction is not yet included in a block. This is why it has no timestamp yet and its block number is listed as `0`.

6. After waiting anywhere from a minute to a few minutes (depending on the current testnet traffic) you should see a confirmation on your transaction.

    *You will need to keep reloading the page for it to update. It doesn't update automatically.*

    ![1 Confirmation Received](Bitcoin_Wallet_Images/1_Confirmation_Received.png)

    There is now a timestamp and a block number!

    *As time passes more confirmations will accumulate. You can check on the `confirmations` again at the end of the lab to show yourself this (Given enough time has passed for one or more additional confirmations to occur).*

### Step 8.4 Returning Bitcoins

Since you are feeling charitable you decide to transfer your testnet value back to the faucet's address.

1. Go back to [the faucet site](https://bitcoinfaucet.uo1.net/send.php) and get its bitcoin address

    ![Faucet Address](Bitcoin_Wallet_Images/testnet_faucet_address.png)

    *This address is here for easy copy and paste (ensure this matches what the application shows in the browser before using this, as the bitcoin address they use for returning funds changes occasionally):

    ``` bash
    tb1ql7w62elx9ucw4pj5lgw4l028hmuw80sndtntxt
    ```


2. Send bitcoins to faucet using `Send` section of wallet

    ![Send Bitcoins to Faucet](Bitcoin_Wallet_Images/Send_bitcoins_back_to_faucet.png)

3. Once sent you will see the `Signed transaction` information filled out on your wallet page.

    ![Bitcoins Sent Back to Faucet](Bitcoin_Wallet_Images/bitcoin_send_transaction_output.png)

4. You can go back to the [bitcoin faucet site](https://bitcoinfaucet.uo1.net/send.php) (you may have to refresh the page) and see the transaction as green there with a sender of your address and a transaction ID matching the one you sent.

    ![Bitcoins returned to faucet](Bitcoin_Wallet_Images/transaction_giving_coins_back_to_faucet.png)

5. You can view both the initial received transaction and the newly sent transaction in the `History` tab of your bitcoin wallet.

    *You have to keep reloading the page for it to update with new confirmations. If you wait long enough, eventually these transactions will gain confirmations as in the picture below*

    ![Both transactions with 1 confirmation](Bitcoin_Wallet_Images/both_confirmed_transactions.png)

6. You can continue to play around with the electrum wallet before moving on.

### Summary

You have participated in the bitcoin testnet using a securely built electrum bitcoin wallet powered by a Hyper Protect Virtual Server. Congratulations!!! 

Now, it's time to clean up..

---

## Clean up when you are done with the lab

This section describes important cleanup steps to perform when you have completed the lab. It will guide you through deleting the following artifacts that you created during the lab:

1. The Secure Build Server HPVS instance you created
2. The sample Secure Bitcoin Wallet Application HPVS instance you created
3. The Secure Bitcoin Wallet Application container image that you created for this lab
4. The Docker Access Token that you created for this lab
5. The GitHub ssh key you created for this lab
6. The GitHub personal access token you created for this lab


---

### Step 9.1 List your hpvs instances

Run the code cell below to list your HPVS instances:

In [ ]:
! ibmcloud hpvs instances

---

### Step 9.2 Delete your Secure Build Server HPVS instance

In the output from _Step 9.1_, find your instance named *SBContainer*, and copy its CRN (_Cloud Resource Name_). Paste it at the end of the `ibmcloud hpvs instance-delete ` command in the below code cell and then run it.

**Note:** Make sure that your paste operation leaves a space between the end of your instance's CRN and the _#_ character that begins the comment in the code cell. 

In [ ]:
! ibmcloud hpvs instance-delete --force     # <- paste your full CRN name to the left of the pound sign

---

### Step 9.3 Delete your sample Secure Bitcoin Wallet Application HPVS instance

In the output from _Step 9.1_, find your instance named _securewallet_ and copy its CRN value and paste it at the end of the _ibmcloud hpvs instance-delete_ command in the below code cell and then run it.

In [ ]:
! ibmcloud hpvs instance-delete --force    # <- paste your full CRN name to the left of the pound sign

---

### Step 9.4 Delete your sample secure bitcoin wallet application secure image from Docker Hub

While logged in to your Docker Hub account, follow these steps from the Docker Hub web UI in order to delete the image you created in this lab:

1. Click the *Repositories* link in the banner at the top of the page.
2. Click the image that you created in the lab.
3. Click the _Settings_ link.
4. Scroll down to the _Delete Repository_ section and click the _Delete repository_ button.
5. Enter your image name as confirmation–copy and paste is your friend here– for confirmation and click the _Delete_ button.



---

### Step 9.5 Delete the Docker Access Token you created for this lab

Follow [these instructions](https://docs.docker.com/docker-hub/access-tokens/#modify-existing-tokens) to delete the Docker Access Token you created earlier in the lab.



---

### Step 9.6 Remove the ssh key you created for this lab from your GitHub account

Delete the ssh key you added for the lab from your GitHub account by following steps 1-3 of [these instructions](https://docs.github.com/en/github/authenticating-to-github/reviewing-your-ssh-keys), choosing the _Delete_ button for the ssh key you want to get rid of.  If you were an existing GitHub user prior to this lab, be careful not to delete any other ssh keys you may have added for other purposes.

---

### Step 9.7 Remove the GitHub personal access token you created for this lab from your GitHub account

You should still be logged in to GitHub from the previous step, _Step 9.12_.  Click on your picture or avatar for your account and navigate to _Settings->Developer Settings->Personal access tokens_ and delete the personal access token that you created in _Step 2.5_ of this lab.  Ask an instructor for help if you have difficulty deleting this.

---

## You have reached the end of the lab!

Thank you for taking this lab, and please feel free to provide feedback, either by opening a GitHub issue on the lab repo at https://github.com/ibm-wsc/ibm-cloud-hpvs-sbs-byoi-lab or by sending an email directly to the primary author of the lab, Barry Silliman, at "silliman at us dot ibm dot com"  (the email spammers will never figure out my real email address from that, but you will).

---

## Acknowledgements

My esteemed colleagues Jin VanStee and Garrett Woodworth also made significant contributions to this lab, but I take responsibility for any defects or shortcomings in the lab. -  Barry Silliman

## References

This lab is based largely on publicly available material on IBM's website, especially [Tutorial: Using Secure Build Server with a digital wallet](https://cloud.ibm.com/docs/hp-virtual-servers?topic=hp-virtual-servers-tutorial_secure_build_server)

Additional information about using the Secure Build Server CLI can be found [here](https://github.com/ibm-hyper-protect/secure-build-cli)  

Here is the [GitHub repo for the sample secure bitcoin wallet application](https://github.com/IBM/secure-bitcoin-wallet)